In [6]:
import torch
from torch.autograd import Variable
import  torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# scikit-learn 라이브러리 임포

#Pandas 라이브러리 임포트
import pandas as pd

# Numpy 라이브러리 임포트
import numpy as np

# time 라이브러리 임포트
import time

In [4]:
dataframe_raw = pd.read_csv("./winequality-white.csv",delimiter=";")

In [38]:
def dataframe_to_arrays(dataframe):
    input_cols = list(dataframe.columns)[:-1]
    output_cols=list(dataframe.columns)[-1:]
    dataframe1 = dataframe_raw.copy(deep=True)
    inputs_array = dataframe1[input_cols].to_numpy(dtype=np.float32)
    targets_array = dataframe1[output_cols].to_numpy(dtype=np.longlong).squeeze(1)-3
    return inputs_array, targets_array
inputs_array, targets_array = dataframe_to_arrays(dataframe_raw)
inputs_array.shape, targets_array.shape


((4898, 11), (4898,))

In [48]:
inputs = torch.from_numpy(inputs_array).float()
targets = torch.from_numpy(targets_array).long()

dataset = TensorDataset(inputs, targets )
num_samples = inputs.shape[0]
num_train = int(num_samples*0.8); num_val = num_samples-num_train
train_ds, val_ds = random_split(dataset,[num_train,num_val])
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64)

In [49]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(11, 96)
        self.fc2 = nn.Linear(96, 96)
        self.fc3 = nn.Linear(96, 96)
        self.fc4 = nn.Linear(96, 96)
        self.fc5 = nn.Linear(96, 96)
        self.fc6 = nn.Linear(96, 7)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

model = Net()
        
        

In [51]:
%%time
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01)

for epoch in range(1,1000+1):
    total_loss=0

    for train_x, train_y in train_loader:
        # train_x, train_y = Variable(train_x), Variable(train_y)
        optimizer.zero_grad()
        output = model.forward(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.data

    if(epoch%50)==0:
        total=0; correct=0;
        for inputs, labels in val_loader:
            inputs, labels = inputs, labels
            outputs = model(inputs)
            _, predicted = torch.max(outputs, dim=1)
            for label, p in zip(labels, predicted):
                if(int(label)==int(p)):
                    correct+=1                    
            total+= labels.shape[0]
        print(f"{epoch}:: accuracy: {correct} / {total} = {100.0*correct/total}%")

50:: accuracy: 465 / 980 = 47.44897959183673%
100:: accuracy: 448 / 980 = 45.714285714285715%
150:: accuracy: 470 / 980 = 47.95918367346939%
200:: accuracy: 302 / 980 = 30.816326530612244%
250:: accuracy: 437 / 980 = 44.59183673469388%
300:: accuracy: 436 / 980 = 44.48979591836735%
350:: accuracy: 452 / 980 = 46.12244897959184%
400:: accuracy: 437 / 980 = 44.59183673469388%
450:: accuracy: 459 / 980 = 46.83673469387755%
500:: accuracy: 485 / 980 = 49.48979591836735%
550:: accuracy: 473 / 980 = 48.265306122448976%
600:: accuracy: 443 / 980 = 45.204081632653065%
650:: accuracy: 424 / 980 = 43.265306122448976%
700:: accuracy: 478 / 980 = 48.775510204081634%
750:: accuracy: 499 / 980 = 50.91836734693877%
800:: accuracy: 453 / 980 = 46.224489795918366%
850:: accuracy: 427 / 980 = 43.57142857142857%
900:: accuracy: 469 / 980 = 47.857142857142854%
950:: accuracy: 408 / 980 = 41.63265306122449%
1000:: accuracy: 406 / 980 = 41.42857142857143%
Wall time: 2min 12s


In [47]:
total=0; correct=0;
for inputs, labels in val_loader:
    inputs, labels = inputs, labels
    outputs = model(inputs)
    _, predicted = torch.max(outputs, dim=1)
    for label, p in zip(labels, predicted):
        if(int(label)==int(p)):
            correct+=1                    
    total+= labels.shape[0]
print(f"accuracy: {correct} / {total} = {100.0*correct/total}%")

accuracy: 503 / 980 = 51.326530612244895%
